In [9]:
""" Installing libraries """
%pip install pandas==2.3.2 matplotlib==3.10.6 seaborn==0.13.2 scikit-learn==1.7.1 numpy==2.2.6 pyarrow==21.0.0


Note: you may need to restart the kernel to use updated packages.


In [10]:
""" Importing libraries """

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Our modules

from Modules.read_parquet import read_parquet_file


In [11]:
""" Setting up the constants """

DATA_PATH = "../Data/hackathon_2025_templates/part-00000-tid-"

FILE_HASH = [
    "2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36",
    "5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8",
    "7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1",
]

SUFIX = "-4-1-c000.snappy.parquet"

FILE_NAMES = [DATA_PATH + hash + SUFIX for hash in FILE_HASH]


In [12]:
""" Loading the data """

# for file in FILE_NAMES:

file_path = FILE_NAMES[2]
print(file_path)
first_data = pd.read_parquet(file_path)
# df = read_parquet_file(file_path)
# print(f"Data from {file}:")
# display(df.head())
# print("\n\n")

../Data/hackathon_2025_templates/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet


In [13]:
first_data

,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,2282334733936076502,Distilled Spirits,JOSEPH CARTRON CAFÉ LIQUEUR,Distilled Spirits,Core,Liqueurs & Cordials,Joseph Cartron Cafe,Spiribam
1,6091840953834683482,Distilled Spirits,SPRINGBANK 18 YEAR SINGLE MALT 700ML,Distilled Spirits,Specialty,Scotch Whisky,Springbank 18 Year Single Malt,Pacific Edge Wine & Spirits
2,1968645851245092408,Distilled Spirits,J BRANDT TRIPLE SEC 12/750ML 30PF,Distilled Spirits,Private Label,Liqueurs & Cordials,J Brandt Triple Sec,Sazerac Spirits
3,994706710729219179,Draft,REFORMATION CASHMERE IPA 1/4 KEG,Draft,In&Out,Other Draft,Reformation Cashmere Fresh Hop IPA,Reformation Brewery
4,9209550539540384349,Non-Alcohol,HELLA MOSCOW MULE 750ML,Non Alcohol,Core,Mixers,Hella Bitters Bloody Mary,Hella Bitter Llc
...,...,...,...,...,...,...,...,...
7087,2552936302329139135,Distilled Spirits,BOTANICAL GIN 1 LTR,Distilled Spirits,Close Out,Gin,Botanica Gin,Falcon Spirits
7088,5175418594860080319,Draft,CHIHUAHUA GUAVA LIME 1/6 KEG,Draft,Discontinued,Lager / Pilsner,Chihuahua Guava Lime,Chihuahua Brewing Company
7089,2514391530951310058,Distilled Spirits,TSMOORE PORT FINISH BOURBON 6/750ML 99PF,Distilled Spirits,Allocated,Bourbon Whiskey,Thomas S. Moore Chardonnay Cask Finish Bourbon,Sazerac Spirits
7090,9183313499769355186,Wine,RICH & CREAMY CHARDONNAY 2020,Wine < 14 %,Discontinued,White Wine,Rich & Creamy Chardonnay,Scotto Cellars


In [14]:
print(first_data.shape)
first_data.info()

(7092, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7092 entries, 0 to 7091
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   produto       7092 non-null   object
 1   categoria     7092 non-null   object
 2   descricao     7092 non-null   object
 3   tipos         7092 non-null   object
 4   label         5619 non-null   object
 5   subcategoria  7060 non-null   object
 6   marca         7092 non-null   object
 7   fabricante    7092 non-null   object
dtypes: object(8)
memory usage: 443.4+ KB


In [17]:
# Showing all the categoria_pdv distinct values

# List all columns
cols = first_data.columns.tolist()

for col in cols:
    uniques = first_data[col].nunique()
    col_type = first_data[col].dtype
    # sorted_list = sorted(list(first_data[col].unique()))
    min = 0#sorted_list[0]
    max = 1#sorted_list[-1]
    print(f"|{col}|{col_type}||{min} - {max}| {uniques} distintos|")

|produto|object||0 - 1| 7092 distintos|
|categoria|object||0 - 1| 7 distintos|
|descricao|object||0 - 1| 7092 distintos|
|tipos|object||0 - 1| 22 distintos|
|label|object||0 - 1| 14 distintos|
|subcategoria|object||0 - 1| 42 distintos|
|marca|object||0 - 1| 4221 distintos|
|fabricante|object||0 - 1| 343 distintos|


In [19]:
""" Listing unique values for each column """

for col in cols:
    print(f"Column: {col}")
    # fill empty values with " "
    first_data[col] = first_data[col].fillna(" ")
    # print the unique values

    unique_values = sorted(list(first_data[col].unique()))
    print(f"Unique values ({len(unique_values)}): {unique_values}")
    print("\n\n")

Column: produto
Unique values (7092): ['1000423277513436210', '1000794189319388082', '1000817480846862155', '1004943868572044494', '1005164764874054592', '1006481780268592038', '1009179103632945474', '1010742812441057506', '1010928172993885810', '1011132854339272786', '101132503115441787', '1012020336292689550', '1013000237964729357', '1013934481131638870', '1014303431906261779', '1016862623638068708', '1018842740424501497', '1019923740013333116', '1020026497917953020', '1020804162060274733', '1020862741761856801', '1023169691271142067', '1023598837563683985', '1026171658071894416', '1027530245429186199', '1027752151794664243', '1027821663900970320', '1027867862447166238', '1027944284502633794', '1029370090212151375', '1029546668912065925', '1032600207272444963', '1032904322005411714', '1040821147398770520', '1042792957319957175', '104409766904295795', '104439038099886750', '1044998179813769263', '1047242971096768006', '1049221938490529226', '1050953702696653910', '105461563608081868',